In [ ]:
import os
from huggingface_hub import login

# Get token from environment variable or prompt user to login
token = os.getenv("HF_TOKEN")
login(token=token, add_to_git_credential=True)

In [ ]:
from transformers import AutoModelForCausalLM

llm_checkpoint = "meta-llama/Llama-3.2-1B"

llm_tokenizer = AutoTokenizer.from_pretrained(llm_checkpoint)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_checkpoint,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm_tokenizer.pad_token = llm_tokenizer.eos_token

In [ ]:
from torch.utils.data import DataLoader

NUM_EXAMPLES = 2

# Create few-shot learning prompt for LLM from traindf
few_shot_prompt = "Here are some examples of condescending and non-condescending text.\n\n"

pos_examples = trdf[trdf["labels"] == 1].sample(NUM_EXAMPLES, random_state=RANDOM_SEED)
neg_examples = trdf[trdf["labels"] == 0].sample(NUM_EXAMPLES, random_state=RANDOM_SEED)

# Add examples to few-shot prompt
for i in range(NUM_EXAMPLES):
    example = pos_examples.iloc[i]
    label = "Yes" if example["labels"] == 1 else "No"
    few_shot_prompt += f"Text: {example['text']}\nAnswer: {label}\n\n"

for i in range(NUM_EXAMPLES):
    example = neg_examples.iloc[i]
    label = "Yes" if example["labels"] == 1 else "No"
    few_shot_prompt += f"Text: {example['text']}\nAnswer: {label}\n\n"

few_shot_prompt += "Now determine if the following text is condescending, answering with one word, strictly 'Yes' or 'No':\n\nText: "
print(few_shot_prompt)


In [ ]:
yes_id = llm_tokenizer.encode("Yes", add_special_tokens=False)[0]
no_id = llm_tokenizer.encode("No", add_special_tokens=False)[0]

esds = Dataset.from_pandas(esdf)

def llm_predict(ds):
    data_loader = DataLoader(ds, batch_size=16)

    all_preds = []

    for batch in data_loader:
        batch["prompt"] = [ few_shot_prompt + text + "\nAnswer: " for text in batch["text"] ]
        inputs = llm_tokenizer(
            batch["prompt"], 
            max_length=MAX_LENGTH,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = llm_model(**inputs)

        logits = outputs.logits[:, -1, :]

        yes_logits = logits[:, yes_id]
        no_logits = logits[:, no_id]
        preds = torch.where(yes_logits > no_logits, 1, 0)
        all_preds.extend(preds.cpu().numpy().tolist())

    return all_preds

preds = llm_predict(esds)

f1 = f1_score(esdf["labels"], preds, pos_label=1)
print(f"F1 Score: {f1}")